In [21]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

#                         Table of Contents                          

   1. Basic autograd example 1               
   2. Basic autograd example 2               
   3. Loading data from numpy              
   4. Input pipline                          
   5. Input pipline for custom dataset      
   6. Pretrained model                       
   7. Save and load model                     



## 1. Basic autograd example 1               


In [24]:
# Create tensors.
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

# Compute gradients.
y.backward()

# Print out the gradients.
print(x.grad)    # x.grad = 2
print(w.grad)    # w.grad = 1
print(b.grad)    # b.grad = 1

tensor(2.)
tensor(1.)
tensor(1.)


## 2. Basic autograd example 2              


In [25]:


# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# Build a fully connected layer.
linear = nn.Linear(3, 2)
print ('w: ', linear.weight)
print ('b: ', linear.bias)

# Build loss function and optimizer.
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# Forward pass.
pred = linear(x)

# Compute loss.
loss = criterion(pred, y)
print('loss: ', loss.item())

# Backward pass.
loss.backward()

# Print out the gradients.
print ('dL/dw: ', linear.weight.grad)
print ('dL/db: ', linear.bias.grad)

# 1-step gradient descent.
optimizer.step()

# You can also perform gradient descent at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())


w:  Parameter containing:
tensor([[ 0.1135, -0.0547, -0.3174],
        [-0.2182, -0.3516,  0.4796]], requires_grad=True)
b:  Parameter containing:
tensor([0.3177, 0.2019], requires_grad=True)
loss:  1.2774951457977295
dL/dw:  tensor([[-0.2909,  0.4944,  0.0480],
        [ 0.0765, -0.4988,  0.3831]])
dL/db:  tensor([ 0.5907, -0.0874])
loss after 1 step optimization:  1.2666889429092407


In [28]:
# Create a numpy array.
x = np.array([[1, 2], [3, 4]])
print("x numpy : " ,x)

# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)
print("y torch : " ,y)

# Convert the torch tensor to a numpy array.
z = y.numpy()
print("z numpy : " ,z)



x numpy :  [[1 2]
 [3 4]]
y torch :  tensor([[1, 2],
        [3, 4]])
z numpy :  [[1 2]
 [3 4]]


## 4. Input pipline

In [30]:
# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='data/',
                                             train=True,
                                             transform=transforms.ToTensor(),
                                             download=True)

# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print (image.size())
print (label)

# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64,
                                           shuffle=True)


torch.Size([3, 32, 32])
6


## 5. Input pipline for custom dataset

In [34]:
import torch
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms

# Define transforms (convert to tensor and normalize)
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=(0.5,), std=(0.5,))  # Normalize to [-1, 1]
])

# Load the MNIST dataset (train data)
train_dataset = torchvision.datasets.MNIST(
    root='./data',  # Directory to store the dataset
    train=True,     # Load the training dataset
    download=True,  # Download the dataset if it's not already downloaded
    transform=transform  # Apply the transformation
)

# Create a DataLoader for batch processing
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

# Load one batch of data (for example)
data_iter = iter(train_loader)
images, labels = next(data_iter)  # Correct usage of next()

print(f"Batch of images size: {images.size()}")
print(f"Batch of labels size: {labels.size()}")



Batch of images size: torch.Size([64, 1, 28, 28])
Batch of labels size: torch.Size([64])


## 6. Pretrained model   

In [37]:
# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.

# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size())     # (64, 100)

torch.Size([64, 100])


## Save and load model

In [40]:
# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL torchvision.models.resnet.ResNet was not an allowed global by default. Please use `torch.serialization.add_safe_globals([ResNet])` or the `torch.serialization.safe_globals([ResNet])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by default with w